# Análisis de datos de autos

En este proyecto se plantea la hipótesis de que las características técnicas y de uso de los automóviles, tales como la potencia, la eficiencia de combustible, las emisiones de CO2 y el tipo de tracción, influyen significativamente en su precio, popularidad y demanda en el mercado. Se parte de la premisa de que los autos con mayor eficiencia y menor impacto ambiental presentan una mayor aceptación en mercados actuales, mientras que la potencia y el tipo de tracción pueden estar asociados a segmentos específicos de usuarios.

Para validar estas hipótesis, se utiliza un dataset compuesto por variables como 'Brand', 'Model', 'Year', 'Country', 'Condition', 'Engine_Size', 'Horsepower', 'Torque', 'Weight', 'Top_Speed', 'Acceleration_0_100', 'Fuel_Type', 'Drivetrain', 'Transmission', 'Fuel_Efficiency', 'CO2_Emissions', 'Price', 'Mileage', 'Popularity', 'Safety_Rating', 'Number_of_Owners', 'Market_Demand', 'Insurance_Cost', 'Production_Units', 'Log_Price', 'Log_Mileage', 'decada' y 'Grupo_Potencia'. El objetivo principal es analizar la relación entre estas variables y extraer patrones que permitan comprender mejor el comportamiento del mercado automotriz, así como identificar oportunidades de optimización en la oferta de vehículos.

Durante el análisis exploratorio, se identificaron valores perdidos en varias columnas del dataset, lo que requirió la aplicación de técnicas de limpieza y tratamiento de datos para asegurar la calidad y robustez de los resultados obtenidos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn


In [ ]:
df=pd.read_csv('../data/datasetautostop.csv')
display(df.head())

In [ ]:
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

In [ ]:
def agrupar_por_decada(df):
    """
    Agrega una columna 'decada' al DataFrame agrupando los años en décadas.
    """
    # Asegurarse de que 'Year' es tipo datetime
    if not np.issubdtype(df['Year'].dtype, np.datetime64):
        df['Year'] = pd.to_datetime(df['Year'], format='%Y')
    # Extraer el año como entero
    years = df['Year'].dt.year
    # Calcular la década
    df['decada'] = (years // 10) * 10
    df['decada'] = df['decada'].astype(str) + "s"
    return df

# Aplicar la función al DataFrame
df = agrupar_por_decada(df)
display(df[['Year', 'decada']].head())

In [ ]:
# Informe de calidad de datos del DataFrame
def informe_calidad_datos(df):
    print('--- INFORME DE CALIDAD DE DATOS ---')
    print('\nDimensiones del DataFrame:')
    print(f'Filas: {df.shape[0]}, Columnas: {df.shape[1]}')
    print('\nTipos de variables:')
    print(df.dtypes)
    print('\nValores nulos por columna:')
    print(df.isnull().sum())
    print('\nPorcentaje de valores nulos por columna:')
    print((df.isnull().mean()*100).round(2))
    print('\nCantidad de filas duplicadas:')
    print(df.duplicated().sum())
    print('\nEstadísticas descriptivas (numéricas):')
    print(df.describe().T)
    print('\nEstadísticas descriptivas (categóricas):')
    print(df.describe(include=['object', 'category']).T)
    print('\nValores únicos por columna:')
    print(df.nunique())
    print('\n--- FIN DEL INFORME ---')

# Ejecutar el informe de calidad de datos
informe_calidad_datos(df)

In [ ]:
# Gráfico de barras: cantidad de autos por década usando solo las columnas permitidas
conteo_decadas = df['decada'].value_counts().sort_index()
plt.figure(figsize=(10,6))
bars = conteo_decadas.plot(kind='bar', color='skyblue')
plt.xlabel('Década')
plt.ylabel('Cantidad de autos')
plt.title('Cantidad de autos creados por década')
plt.xticks(rotation=45)

# Agregar los valores encima de cada barra
for i, valor in enumerate(conteo_decadas.values):
    plt.text(i, valor + 10, str(valor), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras horizontales: cantidad de autos por tipo de tracción
conteo_traccion = df['Drivetrain'].value_counts().sort_values()
plt.figure(figsize=(10,6))
conteo_traccion.plot(kind='barh', color='lightcoral')
plt.xlabel('Cantidad de autos')
plt.ylabel('Tipo de tracción')
plt.title('Cantidad de autos por tipo de tracción')

# Agregar los valores al final de cada barra
for i, valor in enumerate(conteo_traccion.values):
    plt.text(valor + 5, i, str(valor), va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
def clasificar_potencia(df):
    """
    Crea una columna 'Grupo_Potencia' con tres categorías según la columna 'Horsepower':
    - 'Potencia baja' <= 593
    - 'Potencia media' <= 1057
    - 'Potencia alta' > 1057
    """
    condiciones = [
        df['Horsepower'] <= 593,
        (df['Horsepower'] > 593) & (df['Horsepower'] <= 1057),
        df['Horsepower'] > 1057
    ]
    opciones = ['Potencia baja', 'Potencia media', 'Potencia alta']
    df['Grupo_Potencia'] = np.select(condiciones, opciones, default='Desconocido')
    return df

# Aplicar la función al DataFrame
df = clasificar_potencia(df)
display(df[['Horsepower', 'Grupo_Potencia']].head())

In [ ]:
# Gráfico de líneas: Fuel_Efficiency por década y grupo de potencia
plt.figure(figsize=(12,7))
for grupo in ['Potencia baja', 'Potencia media', 'Potencia alta']:
    datos = df[df['Grupo_Potencia'] == grupo]
    promedio = datos.groupby('decada')['Fuel_Efficiency'].mean().sort_index()
    plt.plot(promedio.index, promedio.values, marker='o', label=grupo)

plt.xlabel('Década')
plt.ylabel('Fuel Efficiency')
plt.title('Eficiencia de combustible por década y grupo de potencia')
plt.legend(title='Grupo de Potencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de líneas: CO2_Emissions por década y grupo de potencia
plt.figure(figsize=(12,7))
for grupo in ['Potencia baja', 'Potencia media', 'Potencia alta']:
    datos = df[df['Grupo_Potencia'] == grupo]
    promedio = datos.groupby('decada')['CO2_Emissions'].mean().sort_index()
    plt.plot(promedio.index, promedio.values, marker='o', label=grupo)

plt.xlabel('Década')
plt.ylabel('CO2 Emissions')
plt.title('Emisiones de CO2 por década y grupo de potencia')
plt.legend(title='Grupo de Potencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()